In [9]:
import pymmcore
import os.path
import sys
import time

In [2]:
mm_dir = r"C:\Program Files\Micro-Manager-2.0.1"

mmc = pymmcore.CMMCore()

In [3]:
pymmcore.CMMCore().getAPIVersionInfo()

'Device API version 70, Module API version 10'

In [4]:
mmc.setDeviceAdapterSearchPaths([mm_dir])
mmc.loadSystemConfiguration(os.path.join(mm_dir, "MMConfig_xyz_shutter_filter2.cfg"))

# mmc.snapImage()
# mmc.getImage()

In [5]:
mmc.getLoadedDevices()

('COM1',
 'COM2',
 'COM8',
 'PIZStage',
 'XYStage',
 'SutterHub',
 'Wheel-A',
 'Shutter-A',
 'Core')

# PyMMCore control

## get/set XYZ

In [14]:
# XY stage
mmc.setXYPosition(-5000.00,465.00)
mmc.getXYPosition()

[-4999.99, 464.99]

In [15]:
# Z piezo
mmc.setPosition(11)
mmc.getPosition()

11.0677

## get/set shutter/filter wheel position

### filter wheel

In [27]:
for prop in mmc.getDevicePropertyNames("Wheel-A"):
    print(prop, ": ",  mmc.getProperty("Wheel-A", prop))

Busy :  0
ClosedPosition :  
Delay_ms :  0.0000
Description :  Sutter Lambda Filter Wheel
Label :  Filter-2
Name :  Wheel-A
State :  2
State Change Speed :  3


In [19]:
wheel_values = mmc.getStateLabels("Wheel-A")
wheel_values

('Filter-0',
 'Filter-1',
 'Filter-2',
 'Filter-3',
 'Filter-4',
 'Filter-5',
 'Filter-6',
 'Filter-7',
 'Filter-8',
 'Filter-9')

In [18]:
mmc.setStateLabel("Wheel-A", "Filter-0")
mmc.getStateLabel("Wheel-A")

In [20]:
mmc.setStateLabel("Wheel-A", 'Filter-2')

### shutter

In [24]:
mmc.setShutterOpen(True)
mmc.getShutterOpen()

True

In [25]:
mmc.setShutterOpen(False)
mmc.getShutterOpen()

False

# Insight control

In [30]:
import pyvisa

In [33]:
rm = pyvisa.ResourceManager()

resource_list = rm.list_resources()

print(resource_list)

('TCPIP0::140.247.108.15::inst0::INSTR', 'ASRL1::INSTR', 'ASRL2::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL7::INSTR', 'ASRL8::INSTR', 'ASRL11::INSTR')


In [75]:
with rm.open_resource(resource_list[-1]) as my_instrument:
    my_instrument.baud_rate = 115200
    my_instrument.write_termination = '\r'
    # my_instrument.write("*IDN?")
    print(my_instrument.query_ascii_values(r"MODE?", converter="s")[0])
    print(my_instrument.query_ascii_values(r"SHUTter?", converter="s")[0])
    print(my_instrument.query_ascii_values(r"READ:POWer?", converter="s")[0])
    
    # my_instrument.write_ascii_values(r"SHUTter 0", "")
    # my_instrument.write_ascii_values(r"OFF", "")
    time.sleep(1)
    print(my_instrument.query_ascii_values(r"READ:POWer?", converter="s")[0])
    
#     while True:
#         print(my_instrument.read_bytes(1))


RUN

0

2.02

0.00



# ThorLabs Kinesis motorized mounts

In [2]:
from pylablib.devices import Thorlabs

In [16]:
with Thorlabs.KinesisMotor("27004338") as hwp_sample:
    # print(hwp_sample.get_full_status())
    # print(hwp_sample.is_homed())
    # # print(hwp_sample.get_homing_parameters())
    # # time.sleep(3)
    # hwp_sample.home()
    # hwp_sample.wait_for_home()
    # hwp_sample.move_by(1000)
    hwp_sample.move_to(100000)
    

# bh trigger

In [23]:
import nidaqmx
import nidaqmx.system
from nidaqmx.types import CtrTime

In [24]:
system = nidaqmx.system.System.local()

In [25]:
system.driver_version

DriverVersion(major_version=18, minor_version=5, update_version=0)

In [26]:
for device in system.devices:
    print(device)

Device(name=Dev2)


In [29]:
# https://stackoverflow.com/questions/49951166/nidaqmx-co-channels-cant-write-sample
# https://github.com/ni/nidaqmx-python/blob/master/nidaqmx/_task_modules/co_channel_collection.py#L160

In [34]:
with nidaqmx.Task() as task:
    task.co_channels.add_co_pulse_chan_time(counter="Dev2/ctr1", low_time=0.010, high_time=0.010)
    task.start()